In [2]:
#鐵達尼生存率預測中，試著變更門檻值，觀察預測能力是否提升?
library(magrittr)
library(caret)

train=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\titanic_train.csv')
str(train)
train=train[,-c(1,4,9)]

##extract categorical variable
col.type=rep(0,9)
for (i in 1:9) {
  col.type[i]=class(train[,i])
}
col.factor=which(col.type=='factor')
train.num=train[,-col.factor]

## Age NA --> mean 
age.na.loc=which(is.na(train.num$Age)==T)
train.num$Age[age.na.loc]=mean(train.num$Age,na.rm = T)

##only keep categorical column
train.cate=train[,c(col.factor,1)]


## categorical~numeric relationship -->anova
f.value=rep(0,5)
for (i in 2:6) {
  re=aov(unlist(train.num[i])~as.factor(train$Survived))
  re %<>% summary()%>% unlist() %>% as.numeric
  f.value[i-1]=re[9]
}

train.select.f=train.num[,c(1,4,5)]
train.select.f=cbind(train.select.f,Survived=train$Survived)

## regression 
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
reg.cv1= train(as.factor(Survived)~., data=train.num, trControl=train_control, family=binomial,method="glm")
t12=Sys.time()
time1=difftime(t11,t12,units='secs')
acc1=reg.cv1$results[2]
kappa1=reg.cv1$results[3]


## xgb
t11=Sys.time()
xgb1=train(as.factor(Survived)~., data=train.num, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.acc1=mean(xgb1$results$Accuracy)
gb.kappa1=mean(xgb1$results$Kappa)
t12=Sys.time()
time.gb1=difftime(t11,t12,units='secs')

## regression + p.value < 0.015
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
reg.cv2= train(as.factor(Survived)~., data=train.select.f, trControl=train_control, family=binomial,method="glm")
t12=Sys.time()
time2=difftime(t11,t12,units='secs')
acc2=reg.cv2$results[2]
kappa2=reg.cv2$results[3]


## xgb + p.value < 0.015
t11=Sys.time()
xgb2=train(as.factor(Survived)~., data=train.select.f, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.acc2=mean(xgb2$results$Accuracy)
gb.kappa2=mean(xgb2$results$Kappa)

time.gb2=difftime(t11,t12,units='secs')



#續上題，使用 L1 Embedding 做特徵選擇(自訂門檻)，觀察預測能力是否提升?
tuneGrid = expand.grid(alpha = 1,
                       lambda = 0)
t11=Sys.time()
lasso.cv1 = train(as.factor(Survived)~.,
                  data = train.num, 
                  method = "glmnet", 
                  trControl = train_control,
                  tuneGrid =tuneGrid,
                  family='binomial')
t12=Sys.time()

time.lasso=difftime(t11,t12,units='secs')

lass.acc=lasso.cv1$results$Accuracy
lass.kappa=lasso.cv1$results$Kappa

result1=data.frame(type=c('reg','reg+ high cor','xgb','xgb+ high cor','lasso'),
                   acc=as.numeric(c(acc1,acc2,gb.acc1,gb.acc2,lass.acc)),
                   kappa=as.numeric(c(kappa1,kappa2,gb.kappa1,gb.kappa2,lass.kappa)),
                   time=-c(time1,time2,time.gb1,time.gb2,time.lasso))
result1

ERROR: Error in library(e1071): there is no package called 'e1071'


### (1) anova 之p. value 所挑出的3個feature所建立的模型，其表現並沒有比較好。
### (2) lasso的結果亦沒有較好。